<a href="https://colab.research.google.com/github/lltera/programmers_kdt_II/blob/main/2%EC%9D%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B5_%EC%95%A0%ED%94%8C_%EC%A3%BC%EC%8B%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

애플 주식 데이터를 가지고 간단한 데이터 분석을 해보자. 모든 답은 Pyspark을 통해 이뤄져야 한다.

먼저 PySpark과 Py4J를 설치하자

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 29 kB/s 
     |████████████████████████████████| 198 kB 19.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=25bb146f349902a9fde46bf1ba3b4c830f862abb951540ecf6f4cfe4fe368a62
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [17]:
!ls -t1 sample_data

mnist_test.csv
mnist_train_small.csv
california_housing_test.csv
california_housing_train.csv
anscombe.json
README.md


In [16]:
!ls -t1

sample_data


#### Spark Session 만들기

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Dataframe basic example") \
    .getOrCreate()

#### 애플 주식 CSV 파일 로딩하기: https://pyspark-test-sj.s3-us-west-2.amazonaws.com/appl_stock.csv
일단 pandas 데이터프레임으로 로딩해서 Spark 데이터프레임으로 변경한다

In [3]:
import pandas as pd

apple_pandas_df = pd.read_csv("https://pyspark-test-sj.s3-us-west-2.amazonaws.com/appl_stock.csv")
apple_spark_df = spark.createDataFrame(apple_pandas_df)

#### 1> 어떤 컬럼 이름들이 있는가?

In [4]:
apple_spark_df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### 2> 스키마를 프린트해보기

In [5]:
apple_spark_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Adj Close: double (nullable = true)



#### 3> 처음 5개의 레코드를 출력해보기

In [6]:
apple_spark_df.show(n=5)

+----------+----------+----------+----------+----------+---------+------------------+
|      Date|      Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+----------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.380001|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|213.249994|214.379993|150476200|         27.774976|
|2010-01-06|214.379993|    215.23|210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|209.050005|    210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.060005|211.980005|111902700|         27.464034|
+----------+----------+----------+----------+----------+---------+------------------+
only showing top 5 rows



#### 4> describe를 사용하여 데이터프레임의 컬럼별 통계보기

In [7]:
apple_spark_df.describe().show()

+-------+----------+------------------+-----------------+------------------+------------------+--------------------+------------------+
|summary|      Date|              Open|             High|               Low|             Close|              Volume|         Adj Close|
+-------+----------+------------------+-----------------+------------------+------------------+--------------------+------------------+
|  count|      1762|              1762|             1762|              1762|              1762|                1762|              1762|
|   mean|      null|313.07631115890996|315.9112880164584|309.82824050794557| 312.9270656379114| 9.422577587968218E7| 75.00174115607273|
| stddev|      null|185.29946803981545|186.8981768648577|183.38391664370988|185.14710361709427|6.0205187765927084E7|28.574929721799037|
|    min|2010-01-04|              90.0|        90.699997|         89.470001|         90.279999|            11475900|         24.881912|
|    max|2016-12-30|        702.409988|       70

#### 5> Close 컬럼의 평균값은 얼마인가?

In [8]:
from pyspark.sql.functions import mean

apple_spark_df.select(mean("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|312.9270656379114|
+-----------------+



#### 6> Volume 컬럼의 최대값과 최소값은?

In [9]:
from pyspark.sql.functions import min, max

apple_spark_df.select(max("Volume"), min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|  470249500|   11475900|
+-----------+-----------+



#### 보너스 질문: HV ratio라는 이름의 새로운 컬럼을 추가한 데이터프레임을 만들기. 이 컬럼의 값은 High/Volume으로 계산된다

In [10]:
apple_spark_df_with_hv = apple_spark_df.withColumn("hv ratio", apple_spark_df.High/apple_spark_df.Volume) 

In [11]:
apple_spark_df_with_hv.show(5)

+----------+----------+----------+----------+----------+---------+------------------+--------------------+
|      Date|      Open|      High|       Low|     Close|   Volume|         Adj Close|            hv ratio|
+----------+----------+----------+----------+----------+---------+------------------+--------------------+
|2010-01-04|213.429998|214.499996|212.380001|214.009998|123432400|         27.727039|1.737793286041590...|
|2010-01-05|214.599998|215.589994|213.249994|214.379993|150476200|         27.774976|1.432718223878593...|
|2010-01-06|214.379993|    215.23|210.750004|210.969995|138040000|27.333178000000004|1.559185743262822...|
|2010-01-07|    211.75|212.000006|209.050005|    210.58|119282800|          27.28265|1.777288980473295...|
|2010-01-08|210.299994|212.000006|209.060005|211.980005|111902700|         27.464034|1.894503045949740...|
+----------+----------+----------+----------+----------+---------+------------------+--------------------+
only showing top 5 rows



#### 보너스 질문: 월별 Close 컬럼의 평균값은?

In [12]:
from pyspark.sql.functions import month

monthdf = apple_spark_df.withColumn("Month", month("Date"))

In [13]:
monthavgdf = monthdf.select(["Month", "Close"]).groupBy("Month").mean()

In [14]:
monthavgdf.show()

+-----+----------+------------------+
|Month|avg(Month)|        avg(Close)|
+-----+----------+------------------+
|   12|      12.0|302.35053626845644|
|    1|       1.0| 322.2097142571429|
|    6|       6.0|288.12546566000003|
|    3|       3.0| 332.9115673137254|
|    5|       5.0|  351.621020857143|
|    9|       9.0| 301.0763195902777|
|    4|       4.0| 340.5104108150685|
|    8|       8.0|300.43858096129026|
|    7|       7.0| 281.7221621148649|
|   10|      10.0|308.30552563157903|
|   11|      11.0| 306.2725174895105|
|    2|       2.0| 321.3595563037037|
+-----+----------+------------------+



In [15]:
monthavgdf.select(["Month", "avg(Close)"]).orderBy("Month").show()

+-----+------------------+
|Month|        avg(Close)|
+-----+------------------+
|    1| 322.2097142571429|
|    2| 321.3595563037037|
|    3| 332.9115673137254|
|    4| 340.5104108150685|
|    5|  351.621020857143|
|    6|288.12546566000003|
|    7| 281.7221621148649|
|    8|300.43858096129026|
|    9| 301.0763195902777|
|   10|308.30552563157903|
|   11| 306.2725174895105|
|   12|302.35053626845644|
+-----+------------------+



In [20]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

In [21]:
spark

In [27]:
name_list_json = [ '{"name":"youngjin"}','{"name":"june"}','{"name":"jin"}']

In [28]:
for n in name_list_json:
  print(n)

{"name":"youngjin"}
{"name":"june"}
{"name":"jin"}


In [29]:
import json
for n in name_list_json:
  jn = json.loads(n)
  print(jn["name"])

youngjin
june
jin


In [30]:
rdd = spark.sparkContext.parallelize(name_list_json)

In [31]:
rdd

ParallelCollectionRDD[36] at readRDDFromFile at PythonRDD.scala:262

In [32]:
rdd.count()

3

In [33]:
parsed_rdd = rdd.map(lambda el:json.loads(el))

In [34]:
parsed_rdd

PythonRDD[38] at RDD at PythonRDD.scala:53

In [35]:
parsed_rdd.collect()

[{'name': 'youngjin'}, {'name': 'june'}, {'name': 'jin'}]

In [37]:
parsed_name_rdd = rdd.map(lambda el:json.loads(el)["name"])

In [38]:
parsed_name_rdd.collect()

['youngjin', 'june', 'jin']

In [40]:
from pyspark.sql.types import StringType

In [41]:
df = spark.createDataFrame(name_list_json,StringType())

In [42]:
df.count()

3

In [43]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [44]:
df.select('*').collect()

[Row(value='{"name":"youngjin"}'),
 Row(value='{"name":"june"}'),
 Row(value='{"name":"jin"}')]

In [45]:
df.select('value').collect()

[Row(value='{"name":"youngjin"}'),
 Row(value='{"name":"june"}'),
 Row(value='{"name":"jin"}')]

In [46]:
from pyspark.sql import Row

In [47]:
row = Row("name")
df_name = parsed_name_rdd.map(row).toDF()

In [48]:
df_name.select('name').collect()

[Row(name='youngjin'), Row(name='june'), Row(name='jin')]

In [49]:
df_name.printSchema()

root
 |-- name: string (nullable = true)

